In [1]:
# Copyrights (c) Charles Le Losq 2017
import numpy as np
import os, sys

import matplotlib
matplotlib.use('Agg') # non interactive renderer
from matplotlib import pyplot as plt

import pandas as pd
import rampy
import scipy

import pickle

from tqdm import tqdm

In [3]:
start_path =r'./data/To_Recognize/spectra/'
out_path =r'./data/To_Recognize/'

#spectra_liste = pd.read_excel('./data/file_liste_excellent.xlsx',sheet_name='liste') # RRUUFF dataset
choice = pd.read_excel('./data/file_liste_excellent.xlsx',sheet_name='subset_to_train')
spectra_liste = pd.read_excel('./data/file_liste_excellent.xlsx',sheet_name='testnat') # anto's zircon and diamonds

print('Preparing to treat the spectra in\n'+start_path+'\nand output the images in path\n'+out_path)

# getting options
nb_class = 27 # Switching between the plot or xy mode
switch_baseline = True # In case the baseline is wanted
delimiter = " " # In case the baseline is wanted

# getting the index
print("Getting the interesting data...")
for j in tqdm(range(len(choice['mineral']))):
    if j == 0:
        idx_liste = spectra_liste.loc[spectra_liste['mineral'] == choice['mineral'][j]].index.values
    else:
        idx_liste2 = spectra_liste.loc[spectra_liste['mineral'] == choice['mineral'][j]].index.values
        idx_liste = np.hstack((idx_liste,idx_liste2))

# For sklearn output
xcommon = np.arange(200,1200,1.0)
output_bulk_array = np.zeros((len(idx_liste),len(xcommon)))
output_labels = []
array_labels = np.zeros((len(idx_liste),nb_class))

print("Processing the interesting data...")
for i in tqdm(range(len(idx_liste))):
    idx = idx_liste[i]
    
    # Data importation, baseline treatment, and normalisation
    spectrum = np.genfromtxt(start_path+spectra_liste['filename'][idx],delimiter=delimiter,skip_header=10)

    if switch_baseline == True:
        out1, out2 = rampy.baseline(spectrum[:,0],spectrum[:,1],np.array([[0,1600]]),"als",lam=10.0**5)
        x = spectrum[:,0]
        y = out1[:,0]
    else:
        x = spectrum[:,0]
        y = spectrum[:,1]

    # For the X output = the spectrum
    f = scipy.interpolate.interp1d(x,y,bounds_error=False,fill_value=0.0) # y values outside the observed x are put to 0.
    output_bulk_array[i,:] = f(xcommon)/np.max(f(xcommon))

    # For the Y output = the labels
    label = int(choice[choice['mineral'] == spectra_liste['mineral'][idx]]['label'])
    output_labels.append(label)
    array_labels[i,label] = 1

 11%|█         | 1/9 [00:00<00:01,  7.25it/s]

Preparing to treat the spectra in
./data/To_Recognize/spectra/
and output the images in path
./data/To_Recognize/
Getting the interesting data...
Processing the interesting data...


100%|██████████| 9/9 [00:00<00:00,  9.04it/s]


In [4]:
# in case we ask output for sklearn, we need to save it
with open(out_path+'labels.pkl', 'wb') as fp:
        pickle.dump(output_labels, fp)
np.save(out_path+'array_labels',array_labels)
np.save(out_path+'obs', output_bulk_array)

print('Done')

Done
